<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/PySparkLogisticRegChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 31.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=93eedd4183f70382b4a1787d226a7b169ef1244579242daec6e4b37533fe01d2
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Customer Churn means customer will stop buying the service

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()

In [6]:
from pyspark.sql import functions as sf

In [7]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)


In [8]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [10]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[ 'Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol = 'features')

In [14]:
output = assembler.transform(data)
final_data = output.select('features','churn')

In [15]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')

In [18]:
fitted_churn_model = lr_churn.fit(train_churn)
training_summary = fitted_churn_model.summary

In [19]:
training_summary.predictions.describe().show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               656|                656|
|   mean|0.1524390243902439|0.10823170731707317|
| stddev| 0.359720199920017| 0.3109098904348499|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [22]:
evaluate_labels = fitted_churn_model.evaluate(test_churn)
evaluate_labels.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.81301029325386...|[0.99194208826049...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.37097380480602...|[0.99537191743782...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.49705679024555...|[0.81713509653522...|       0.0|
|[28.0,11128.95,1....|    0|[4.07132619301655...|[0.98323123164094...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.66705773419085...|[0.93505458346895...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.57650645473011...|[0.98981403666405...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.68884564264661...|[0.97560895152858...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.33947538654346...|[0.96575849809229...|       0.0|
|[30.0,10960.52,1....|    0|[2.33362507007510...|[0.91162382962627...|       0.0|
|[31.0,5387.75,0

In [24]:
# check area under the curve 

from pyspark.ml.evaluation import BinaryClassificationEvaluator

churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')
# auc - area under curve 
auc = churn_eval.evaluate(evaluate_labels.predictions)
auc

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


0.7468041237113402


# predict on new data


In [25]:
# fit this model with all hyperparam learned above with whole data (hyperparm has been left to default in this case )
final_lr_model = lr_churn.fit(final_data)

In [26]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [27]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [28]:
new_customers_transformed = assembler.transform(new_customers)
new_customers_transformed.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
result_new_customers = final_lr_model.transform(new_customers_transformed)

In [31]:
result_new_customers.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



these 4 customers are going to churn 

